# ML-Leaks Adversary 3

## Imports

In [1]:
import sys 
import numpy as np 
import matplotlib.pyplot as plt


import torch
import torchvision 
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data.sampler import SubsetRandomSampler

sys.path.insert(0, '../../../Utils/')

import models
from train import *
from metrics import * 

#audio
import librosa as libr
from torch.utils.data import Dataset, DataLoader

print("Python: %s" % sys.version)
print("Pytorch: %s" % torch.__version__)

# determine device to run network on (runs on gpu if available)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

Python: 3.6.5 (default, Jul  6 2018, 19:12:46) 
[GCC 5.4.0 20160609]
Pytorch: 0.4.0


## Hyperparameters

In [2]:
n_epochs = 25
batch_size = 256
lr = 0.001



## Load LibriSpeech

In [3]:
### Hyperparameters

n_seconds = 3
sampling_rate = 16000
number_of_mels =128
all_data = ['train-clean-100']
lr = 0.001

### Speech preprocessing

class tensorToMFCC:
    def __call__(self, y):
#         y = y.numpy()
        dims = y.shape
        y = libr.feature.melspectrogram(np.reshape(y, (dims[1],)), 16000, n_mels=number_of_mels,
                               fmax=8000)
        y = libr.feature.mfcc(S = libr.power_to_db(y))
        y = torch.from_numpy(y)                           
        return y.float()
    
    
class STFT:
    def __call__(self,y):
        dims = y.shape
        y = np.abs(libr.core.stft(np.reshape(y, (dims[1],))))
        y = torch.from_numpy(y).permute(1,0)
        return y.float()
    
    
transform  = STFT()

### Data set

%load_ext autoreload
%autoreload 2
sys.path.insert(0, './../../../Utils')
from datasets import LibriSpeechDataset
from datasets import Libri_preload_and_split

path = './../../../Classification_baselines/LibriSpeech/data/'

splits = [0.8, 0.2] #input fraction of data you want partitioned. Train, test fraction
attacking = 3 #0 for no attack, 1 for attack 1, 3 for attack 3
# attacking means data for a target & shadow network.
# This will also split "out data" from totally different speakers -- data none of the 
# other networks have seen, for training & testing the attack network. This will be
# an equivalent amount of data to the train split as defined about

if sum(splits) != 1:
    print('error: splits do not sum to 1.')

#Splits data into 2 sets of speakers for target & shadow network, into above defined train:test splits
dfs = Libri_preload_and_split(path,all_data,n_seconds,pad=False,cache=True,splits=splits, attacking = attacking)  

#target train & test
valid_sequence_train = LibriSpeechDataset(path, df = dfs[0], seconds = n_seconds, downsampling=1, 
                                    transform = transform, stochastic=False)

valid_sequence_test = LibriSpeechDataset(path, df = dfs[1], seconds = n_seconds, downsampling=1, 
                                    transform = transform, stochastic=False)

#shadow train & test

valid_sequence_out = LibriSpeechDataset(path, df = dfs[2], seconds = n_seconds, downsampling=1, 
                                    transform = transform, stochastic=False)


Initialising LibriSpeechDataset with minimum length = 3s and subsets = ['train-clean-100']
Finished indexing data. 27949 usable files found.
12407
3170
12372
0
Finished splitting data.


In [4]:
# Loaders for data for target model & shadow model 
target_train_loader = DataLoader(valid_sequence_train,
                      batch_size=32,
                      shuffle=True,
                      num_workers=8
                     # pin_memory=True # CUDA only
                     )

target_out_loader = DataLoader(valid_sequence_out,
                      batch_size=32,
                      shuffle=True,
                      num_workers=8
                     # pin_memory=True # CUDA only
                     )

target_test_loader = DataLoader(valid_sequence_test,
                      batch_size=32,
                      shuffle=True,
                      num_workers=8
                     # pin_memory=True # CUDA only
                     )

## ML Leaks Adversary 3 (Membership Inference Attack) 
### https://arxiv.org/abs/1806.01246

In [5]:
# the model being attacked (architecture can be different than shadow)
target_net_type = models.STFT_CNN_classifier

in_size = 94# 20 forMFCC,  94 for STFT
n_hidden = 512
n_classes = valid_sequence_test.num_speakers
print(n_classes)

target_net = target_net_type(in_size,n_hidden,n_classes).to(device)
target_net.apply(models.weights_init)

#target_net.avgpool = nn.AvgPool2d(2, stride=1)
#target_net = models.mlleaks_cnn().to(device)

target_loss = nn.CrossEntropyLoss()
target_optim = optim.Adam(target_net.parameters(), lr=lr)

139


In [ ]:
train(target_net, target_train_loader, target_test_loader, target_optim, target_loss, n_epochs, verbose=False)


[0/25]
Training:

Accuracy = 63.32 %


Test:

Accuracy = 50.66 %


[1/25]
Training:

Accuracy = 86.24 %


Test:

Accuracy = 72.93 %


[2/25]
Training:

Accuracy = 92.74 %


Test:

Accuracy = 79.97 %


[3/25]
Training:

Accuracy = 94.17 %


Test:

Accuracy = 81.51 %


[4/25]
Training:

Accuracy = 94.60 %


Test:

Accuracy = 81.64 %


[5/25]
Training:

Accuracy = 96.66 %


Test:

Accuracy = 85.49 %


[6/25]
Training:


In [ ]:
def save_checkpoint(state, is_best, filename='checkpoint.pth.tar'):
    torch.save(state, filename)
    if is_best:
        shutil.copyfile(filename, 'model_best.pth.tar')

In [ ]:
epoch = 24
save_checkpoint({
            'epoch': epoch,
            'arch': 'CNN_voice_classifier',
            'state_dict': target_net.state_dict(),
            'optimizer' : target_optim.state_dict(),
        }, False, filename = 'model_weights/CNN_voice_classifier100STFT_attack3_'+str(epoch)+'.pth')

In [ ]:
eval_membership_inference(target_net, target_train_loader, target_out_loader)

In [ ]:
# #other model:
# in_sizeFT = 24

